In [7]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler

# Load and scale the dataset
digits = load_digits()
A = digits.data
y = digits.target.reshape(-1, 1)

# Standardize the data matrix A
scaler = StandardScaler()
A = scaler.fit_transform(A)

# Define the gradient for Direct OLSLR
def direct_ols_gradient(A, x, y):
    return A.T @ (A @ x - y)

# BFGS Implementation for Direct OLSLR
def bfgs_direct_ols(A, y, tol=1e-6, max_iter=100, epsilon=1e-8, regularization=1e-5):
    n = A.shape[1]
    x = np.zeros((n, 1))  # Starting point
    B = np.eye(n)  # Initial Hessian approximation
    history = []

    # Regularization for improved stability
    regularization_matrix = regularization * np.eye(n)

    for k in range(max_iter):
        grad = direct_ols_gradient(A, x, y)

        # Check for convergence
        if np.linalg.norm(grad) < tol:
            break

        # Regularized Hessian for better stability
        hess_approx = A.T @ A + regularization_matrix

        # Compute search direction
        p = -np.linalg.solve(B, grad)
        x_new = x + p

        s = x_new - x
        y_grad = direct_ols_gradient(A, x_new, y) - grad

        # Safeguard for small denominators
        denom_1 = max(s.T @ y_grad, epsilon)
        denom_2 = max(s.T @ (B @ s), epsilon)

        # Ensure s and y_grad are column vectors
        s = s.reshape(-1, 1)
        y_grad = y_grad.reshape(-1, 1)

        # Update B using the stabilized BFGS formula
        Bs = B @ s
        B += np.outer(s, s) / denom_1 - np.outer(Bs, Bs) / denom_2

        # Update x and save history
        x = x_new
        history.append(np.linalg.norm(grad))

        # Debugging: Print intermediate values
        if k % 10 == 0:  # Print every 10 iterations
            print(f"Iteration {k}, Gradient Norm: {np.linalg.norm(grad)}, Denominator: {denom_1}")

    return x, history

# Solve Direct OLSLR using BFGS
print("Solving Direct OLSLR using BFGS...")
x_star_f, history_f = bfgs_direct_ols(A, y)
print(f"x_star_f:\n{x_star_f}")
print(f"Number of iterations: {len(history_f)}")

# Observations
print("\nObservations:")
print("Direct OLSLR using BFGS with stabilization should now avoid NaN or overflow errors.")

Solving Direct OLSLR using BFGS...
Iteration 0, Gradient Norm: 5376.846925988702, Denominator: [[1.59216584e+11]]
Iteration 10, Gradient Norm: 7.659055707878938e+74, Denominator: [[7.09181732e+160]]
Iteration 20, Gradient Norm: 5.2459882499749e+149, Denominator: [[nan]]
Iteration 30, Gradient Norm: nan, Denominator: [[nan]]
Iteration 40, Gradient Norm: nan, Denominator: [[nan]]
Iteration 50, Gradient Norm: nan, Denominator: [[nan]]
Iteration 60, Gradient Norm: nan, Denominator: [[nan]]
Iteration 70, Gradient Norm: nan, Denominator: [[nan]]
Iteration 80, Gradient Norm: nan, Denominator: [[nan]]
Iteration 90, Gradient Norm: nan, Denominator: [[nan]]
x_star_f:
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan

/var/folders/kq/vv3n1vnx70b7hcbwwrw4m02r0000gn/T/ipykernel_97064/1365957882.py:46: RuntimeWarning: overflow encountered in matmul
  denom_1 = max(s.T @ y_grad, epsilon)
/var/folders/kq/vv3n1vnx70b7hcbwwrw4m02r0000gn/T/ipykernel_97064/1365957882.py:46: RuntimeWarning: invalid value encountered in matmul
  denom_1 = max(s.T @ y_grad, epsilon)


In [9]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler

# Load and scale the dataset
digits = load_digits()
A = digits.data
y = digits.target.reshape(-1, 1)

# Standardize the data matrix A
scaler = StandardScaler()
A = scaler.fit_transform(A)

# Define the gradient for Regularized OLSLR
def regularized_ols_gradient(A, x, y, lam):
    return A.T @ (A @ x - y) + lam * x

# BFGS Implementation for Regularized OLSLR
def bfgs_regularized_ols(A, y, lam, tol=1e-6, max_iter=100, epsilon=1e-8, regularization=1e-5):
    n = A.shape[1]
    x = np.zeros((n, 1))  # Starting point
    B = np.eye(n)  # Initial Hessian approximation
    history = []

    # Regularization for improved stability
    regularization_matrix = regularization * np.eye(n)

    for k in range(max_iter):
        grad = regularized_ols_gradient(A, x, y, lam)

        # Check for convergence
        if np.linalg.norm(grad) < tol:
            break

        # Compute search direction
        p = -np.linalg.solve(B + regularization_matrix, grad)
        x_new = x + p

        s = x_new - x
        y_grad = regularized_ols_gradient(A, x_new, y, lam) - grad

        # Debugging: Print intermediate values
        if k % 10 == 0:
            print(f"Iteration {k}")
            print(f"Gradient Norm: {np.linalg.norm(grad)}")
            print(f"s.T @ y_grad: {s.T @ y_grad}")

        # Safeguard for small denominators
        denom_1 = max(s.T @ y_grad, epsilon)
        denom_2 = max(s.T @ (B @ s), epsilon)

        # Ensure s and y_grad are column vectors
        s = s.reshape(-1, 1)
        y_grad = y_grad.reshape(-1, 1)

        # Update B using the stabilized BFGS formula
        Bs = B @ s
        B += np.outer(s, s) / denom_1 - np.outer(Bs, Bs) / denom_2

        # Update x and save history
        x = x_new
        history.append(np.linalg.norm(grad))

    return x, history

# Solve Regularized OLSLR using BFGS with lambda = 0.001
print("\nSolving Regularized OLSLR using BFGS (lambda = 0.001)...")
lambda_val = 0.001
x_star_f_lambda, history_f_lambda = bfgs_regularized_ols(A, y, lam=lambda_val)
print(f"x_star_f_lambda:\n{x_star_f_lambda}")
print(f"Number of iterations: {len(history_f_lambda)}")


Solving Regularized OLSLR using BFGS (lambda = 0.001)...
Iteration 0
Gradient Norm: 5376.846925988702
s.T @ y_grad: [[1.59213429e+11]]
Iteration 10
Gradient Norm: 4.729104087919709e+74
s.T @ y_grad: [[2.42909221e+160]]
Iteration 20
Gradient Norm: 1.897105965587677e+149
s.T @ y_grad: [[inf]]
Iteration 30
Gradient Norm: nan
s.T @ y_grad: [[nan]]
Iteration 40
Gradient Norm: nan
s.T @ y_grad: [[nan]]
Iteration 50
Gradient Norm: nan
s.T @ y_grad: [[nan]]
Iteration 60
Gradient Norm: nan
s.T @ y_grad: [[nan]]
Iteration 70
Gradient Norm: nan
s.T @ y_grad: [[nan]]
Iteration 80
Gradient Norm: nan
s.T @ y_grad: [[nan]]
Iteration 90
Gradient Norm: nan
s.T @ y_grad: [[nan]]
x_star_f_lambda:
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [na

/var/folders/kq/vv3n1vnx70b7hcbwwrw4m02r0000gn/T/ipykernel_97064/2414925323.py:46: RuntimeWarning: overflow encountered in matmul
  print(f"s.T @ y_grad: {s.T @ y_grad}")
/var/folders/kq/vv3n1vnx70b7hcbwwrw4m02r0000gn/T/ipykernel_97064/2414925323.py:49: RuntimeWarning: overflow encountered in matmul
  denom_1 = max(s.T @ y_grad, epsilon)
/var/folders/kq/vv3n1vnx70b7hcbwwrw4m02r0000gn/T/ipykernel_97064/2414925323.py:49: RuntimeWarning: invalid value encountered in matmul
  denom_1 = max(s.T @ y_grad, epsilon)
/var/folders/kq/vv3n1vnx70b7hcbwwrw4m02r0000gn/T/ipykernel_97064/2414925323.py:50: RuntimeWarning: overflow encountered in matmul
  denom_2 = max(s.T @ (B @ s), epsilon)
/var/folders/kq/vv3n1vnx70b7hcbwwrw4m02r0000gn/T/ipykernel_97064/2414925323.py:50: RuntimeWarning: invalid value encountered in matmul
  denom_2 = max(s.T @ (B @ s), epsilon)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/core/numeric.py:925: RuntimeWarning: overflow encounter